In [9]:
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, attrs={'class': 'sortable'}, header=0);
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
myWorkingDF = df[0]
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
myWorkingDF = myWorkingDF[myWorkingDF.Borough != 'Not assigned']
#More than one neighborhood can exist in one postal code area.
myWorkingDF = myWorkingDF.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
myWorkingDF.loc[myWorkingDF['Neighbourhood']=='Not assigned', 'Neighbourhood']=myWorkingDF['Borough']
myWorkingDF.shape

(103, 3)

In [12]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

geodata = pd.read_csv('http://cocl.us/Geospatial_data')
geodata = geodata.rename(columns={"Postal Code": "Postcode"})
geodata
myNewDF = pd.merge(myWorkingDF, geodata, on='Postcode')
myNewDF.head(8)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
